<a href="https://colab.research.google.com/github/niksinfinity/ML_AI_Notebooks/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving test_catvnoncat.h5 to test_catvnoncat.h5
User uploaded file "test_catvnoncat.h5" with length 616958 bytes


In [18]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import h5py
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input,Dense,Flatten,GlobalAveragePooling2D
from keras.backend.tensorflow_backend import set_session



def read_data(data_path):
    data = h5py.File(data_path)
    test_x = data[('test_set_x')]
    test_y = data[('test_set_y')]
    test_y = np.reshape(test_y,(test_y.shape[0],1))
    test_x = np.resize(test_x,(test_x.shape[0],299,299,3))
    print(test_x.shape)
    print(test_y.shape)
    return (test_x,test_y)

test_x,test_y = read_data('./test_catvnoncat.h5')

input = Input(shape=(299,299,3))
base_model = InceptionV3(input_tensor=input,include_top=False, weights='imagenet')
features = base_model.output
features = GlobalAveragePooling2D()(features)
print(features.shape)
x = Dense(756,activation='relu',name='dense1')(features)
x = Dense(256,activation='relu',name='dense2')(x)
output = Dense(1,activation='sigmoid',name='dense3')(x)

model = Model(input,output)

for layer in base_model.layers:
    layer.trainable = False

Adamoptimizer = Adam(lr=0.00001)
model.compile(optimizer=Adamoptimizer, loss='binary_crossentropy',metrics=['acc'])

# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
# set_session(tf.Session(config=config))

# import keras.backend as K
# cfg = K.tf.ConfigProto(gpu_options={'allow_growth': True,'per_process_gpu_memory_fraction':0.95})
# # K.set_session(K.tf.Session(config=cfg))
# with K.tf.Session(config=cfg) as sess:
#     sess.run(K.tf.global_variables_initializer())
#     sess.run(K.tf.tables_initializer())

model.fit(test_x,test_y,epochs=65)


(50, 299, 299, 3)
(50, 1)
(?, 2048)
Epoch 1/65
50/50 [==============================] - 22s 439ms/step - loss: 0.7412 - acc: 0.3600
Epoch 2/65
50/50 [==============================] - 1s 10ms/step - loss: 0.7045 - acc: 0.4000
Epoch 3/65
50/50 [==============================] - 1s 11ms/step - loss: 0.6815 - acc: 0.5600
Epoch 4/65
50/50 [==============================] - 1s 10ms/step - loss: 0.6576 - acc: 0.7200
Epoch 5/65
50/50 [==============================] - 1s 11ms/step - loss: 0.6403 - acc: 0.6800
Epoch 6/65
50/50 [==============================] - 1s 11ms/step - loss: 0.6374 - acc: 0.7000
Epoch 7/65
50/50 [==============================] - 1s 11ms/step - loss: 0.6295 - acc: 0.6600
Epoch 8/65
50/50 [==============================] - 1s 11ms/step - loss: 0.6153 - acc: 0.6600
Epoch 9/65
50/50 [==============================] - 1s 11ms/step - loss: 0.6118 - acc: 0.6600
Epoch 10/65
50/50 [==============================] - 1s 11ms/step - loss: 0.6034 - acc: 0.6600
Epoch 11/65
50/50 [==

In [48]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import h5py


def read_data(data_path):
    data = h5py.File(data_path)
    test_x = data[('test_set_x')]
    test_y = data[('test_set_y')]
    test_y = np.reshape(test_y,(test_y.shape[0],1))
    test_x = np.resize(test_x,(test_x.shape[0],299,299,3))
    print(test_x.shape)
    print(test_y.shape)
    return (test_x,test_y)

test_x,test_y = read_data('./test_catvnoncat.h5')

tf.reset_default_graph()

images = tf.placeholder(tf.float32, (None, 299, 299, 3),name='modelinput')
module = hub.Module('https://tfhub.dev/google/imagenet/inception_v3/feature_vector/3',trainable=False)
features = module(images)
print(features.shape)
# x = tf.keras.layers.GlobalAveragePooling2D()(features)
x = tf.keras.layers.Dense(756,activation='relu')(features)
x = tf.keras.layers.Dense(512,activation='relu')(x)
x = tf.keras.layers.Dense(512,activation='relu')(x)

x = tf.keras.layers.Dense(512,activation='relu')(x)

# x = tf.keras.layers.Dense(128,activation='relu')(x)
logits = tf.keras.layers.Dense(1,activation='sigmoid')(x)

cross_entropy = tf.losses.sigmoid_cross_entropy(logits=logits, multi_class_labels=test_y)
cross_entropy = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(cross_entropy)

accuracy = tf.metrics.accuracy(labels=test_y,predictions=logits,name='acc')
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.9

with tf.Session() as sess:
    sess.run(tf.tables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    for _ in range(50):
      cost,_,acc = sess.run([cross_entropy,optimizer,accuracy],feed_dict={images:test_x})
      print("Cost -->",cost)
      print("Acc --->",acc[0])


(50, 299, 299, 3)
(50, 1)
(?, 2048)
Cost --> 0.6532617
Acc ---> 0.0
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost --> 0.6532617
Acc ---> 0.66
Cost -->